In [1]:
import numpy as np
from xgbsurv.models.utils import transform, transform_back
from xgbsurv.models.eh_final import eh_likelihood, eh_gradient
from xgbsurv.models.eh_ah_final import ah_likelihood, ah_objective
from xgbsurv.models.eh_aft_final import aft_likelihood, aft_objective
import sys
sys.path.append('/Users/JUSC/Documents/xgbsurv_benchmarking/deep_learning/')
from loss_functions_pytorch import eh_likelihood_torch, eh_likelihood_torch_2, aft_likelihood_torch, ah_likelihood_torch
import torch
import math
torch.set_printoptions(precision=10)
from torch.autograd.functional import hessian

In [2]:
# create data function

def ah_data_1(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([[0.67254925, 0.        ],
       [0.8607798 , 0.        ],
       [0.43557394, 0.        ],
       [0.94059044, 0.        ],
       [0.8446509 , 0.        ],
       [0.23657039, 0.        ],
       [0.7462968 , 0.        ],
       [0.99700767, 0.        ],
       [0.2818277 , 0.        ],
       [0.44495037, 0.        ]]) #.reshape(1,10)
        y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32) #.reshape(1,10)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([[0.67254925, 0.        ],
       [0.8607798 , 0.        ],
       [0.43557394, 0.        ],
       [0.94059044, 0.        ],
       [0.8446509 , 0.        ],
       [0.23657039, 0.        ],
       [0.7462968 , 0.        ],
       [0.99700767, 0.        ],
       [0.2818277 , 0.        ],
       [0.44495037, 0.        ]], requires_grad=True)#.reshape(10,1)
        y = torch.tensor([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]]) #.reshape(10,1)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event

# create data function

def ah_data_2(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([0.67254923,
        0.86077982,
        0.43557393,
        0.94059047,
        0.8446509 ,
        0.23657039,
        0.74629685,
        0.99700768,
        0.28182768,
        0.44495038]) #.reshape(1,10)
        y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32) #.reshape(1,10)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([0.67254923,
        0.86077982,
        0.43557393,
        0.94059047,
        0.8446509 ,
        0.23657039,
        0.74629685,
        0.99700768,
        0.28182768,
        0.44495038], requires_grad=True).reshape(10,1)
        y = torch.tensor([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]]).reshape(10,1)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event

In [3]:
y, linear_predictor, time, event = ah_data_1(type='torch')

## Structure

- Compare loss to original function

## 1. Compare loss to original function



In [4]:
# EH loss from paper

def eaftloss(out, time, delta): ##loss function for AFT or EH
    ia, ib = out.size()
    if ib == 1: ###loss function for AFT
        n = len(delta)
        print('aft')
        h = 1.30*math.pow(n,-0.2)
        #h 1.304058*math.pow(n,-0.2)  ## 1.304058*n^(-1/5) or 1.587401*math.pow(n,-0.333333) 1.587401*n^(-1/3)
        time = time.view(n,1)
        delta = delta.view(n,1)
        
        # R = g(Xi) + log(Oi)
        R = torch.add(out,torch.log(time)) 
        
        # Rj - Ri
        rawones = torch.ones([1,n], dtype = out.dtype)
        R1 = torch.mm(R,rawones)
        R2 = torch.mm(torch.t(rawones),torch.t(R))
        DR = R1 - R2 
        
        # K[(Rj-Ri)/h]
        K = normal_density(DR/h)
        Del = torch.mm(delta, rawones)
        DelK = Del*K 
        
        # (1/nh) *sum_j Deltaj * K[(Rj-Ri)/h]
        Dk = torch.sum(DelK, dim=0)/(n*h)
        
        # log {(1/nh) * Deltaj * K[(Rj-Ri)/h]}    
        log_Dk = torch.log(Dk)     
        A = torch.t(delta)*log_Dk/n   
        S1 = A.sum()  
        
        ncdf=torch.distributions.normal.Normal(torch.tensor([0.0], dtype = out.dtype), torch.tensor([1.0], dtype = out.dtype)).cdf
        P = ncdf(DR/h)
        CDF_sum = torch.sum(P, dim=0)/n
        Q = torch.log(CDF_sum)
        S2 = -(delta*Q.view(n,1)).sum()/n
             
        S0 = -(delta*torch.log(time)).sum()/n
        
        S = S0 + S1 + S2 
        S = -S
    else: ### loss function for Extended hazard model
        print('not aft model')
        n = len(out[:,0])
        h = 1.30*math.pow(n,-0.2)  ## or 1.59*n^(-1/3)
        time = time.view(n,1)
        delta = delta.view(n,1)
        g1 = out[:,0].view(n,1)
        g2 = out[:,1].view(n,1)
        
        # R = g(Xi) + log(Oi)
        R = torch.add(g1,torch.log(time)) 
        
        S1 =  (delta*g2).sum()/n
        S2 = -(delta*R).sum()/n
        
        # Rj - Ri
        rawones = torch.ones(1,n)
        R1 = torch.mm(R,rawones)
        R2 = torch.mm(torch.t(rawones),torch.t(R))
        DR = R1 - R2 
        
        # K[(Rj-Ri)/h]
        K = normal_density(DR/h)
        Del = torch.mm(delta, rawones)
        DelK = Del*K 
        
        # (1/nh) *sum_j Deltaj * K[(Rj-Ri)/h]
        Dk = torch.sum(DelK, dim=0)/(n*h)  ## Dk would be zero as learning rate too large!
        
        # log {(1/nh) * Deltaj * K[(Rj-Ri)/h]}    
        log_Dk = torch.log(Dk)    
        
        S3 = (torch.t(delta)*log_Dk).sum()/n    
        
        # Phi((Rj-Ri)/h)
        ncdf=torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0])).cdf
        P = ncdf(DR/h) 
        L = torch.exp(g2-g1)
        LL = torch.mm(L,rawones)
        LP_sum = torch.sum(LL*P, dim=0)/n
        Q = torch.log(LP_sum)
        
        S4 = -(delta*Q.view(n,1)).sum()/n
        
        S = S1 + S2 + S3 + S4  
        S = -S
    return S

def normal_density(a):  
    b = 0.3989423*torch.exp(-0.5*torch.pow(a,2.0))
    return b

### AH Loss Original Paper

In [5]:
y, linear_predictor, time, event = ah_data_1(type='torch')
aft_loss_paper = eaftloss(linear_predictor, time, event)
print(aft_loss_paper )

not aft model
tensor(1.4591797590, grad_fn=<NegBackward0>)


### AH Loss Pytorch

check bandwidth, check dims vector for aft, go through code step by step

In [6]:
y, linear_predictor, time, event = ah_data_2(type='torch')
ah_loss_own_torch = ah_likelihood_torch(linear_predictor.reshape(-1),y.reshape(-1))
print(ah_loss_own_torch)

tensor(-1.4591798782, grad_fn=<MulBackward0>)


## AH Loss Numba

In [7]:
y, linear_predictor, time, event = ah_data_2(type='np')
ah_loss_own_numba = ah_likelihood(
     y, linear_predictor
)
ah_loss_own_numba

1.4591796657817264

### Loss Function Comparison

In [40]:
np.allclose(ah_loss_own_torch.detach().numpy() ,ah_loss_own_torch.detach().numpy(),ah_loss_own_numba)

True

## Gradient Comparison

In [41]:
# create data function

def ah_data_1(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([[0.67254925, 0.        ],
       [0.8607798 , 0.        ],
       [0.43557394, 0.        ],
       [0.94059044, 0.        ],
       [0.8446509 , 0.        ],
       [0.23657039, 0.        ],
       [0.7462968 , 0.        ],
       [0.99700767, 0.        ],
       [0.2818277 , 0.        ],
       [0.44495037, 0.        ]]) #.reshape(1,10)
        y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32) #.reshape(1,10)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([[0.67254925, 0.        ],
       [0.8607798 , 0.        ],
       [0.43557394, 0.        ],
       [0.94059044, 0.        ],
       [0.8446509 , 0.        ],
       [0.23657039, 0.        ],
       [0.7462968 , 0.        ],
       [0.99700767, 0.        ],
       [0.2818277 , 0.        ],
       [0.44495037, 0.        ]], requires_grad=True)#.reshape(10,1)
        y = torch.tensor([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]]) #.reshape(10,1)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event

# create data function

def ah_data_2(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([0.67254923,
        0.86077982,
        0.43557393,
        0.94059047,
        0.8446509 ,
        0.23657039,
        0.74629685,
        0.99700768,
        0.28182768,
        0.44495038])
        y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([0.67254923,
        0.86077982,
        0.43557393,
        0.94059047,
        0.8446509 ,
        0.23657039,
        0.74629685,
        0.99700768,
        0.28182768,
        0.44495038], dtype=torch.float32, requires_grad=True).reshape(-1)
        y = torch.tensor([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]],dtype=torch.float32).reshape(-1)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event

### Gradient Own Pytorch



In [43]:
y, linear_predictor, time, event = ah_data_2(type='torch')
#linear_predictor, y  = linear_predictor.reshape(-1), y.reshape(-1)
ah_loss_own_torch = ah_likelihood_torch(linear_predictor,y)
print('loss', ah_loss_own_torch)
ah_loss_own_torch.backward()
print(linear_predictor.is_leaf)
#linear_predictor.retain_grad()
grad_torch = linear_predictor.grad
print('grad_torch', grad_torch.numpy())

loss tensor(-1.4591798782, grad_fn=<MulBackward0>)


AttributeError: 'function' object has no attribute 'backward'

## AH Grad Paper

In [ ]:
y, linear_predictor, time, event = ah_data_1(type='torch')
ah_loss_paper = eaftloss(linear_predictor, time, event)
ah_loss_paper.backward()
ah_grad_paper = linear_predictor.grad
ah_grad_paper

not aft model


tensor([[ 0.0295691416, -0.0943050757],
        [ 0.0034586210,  0.0204732958],
        [ 0.0027502086,  0.0190855525],
        [ 0.0024057813,  0.0286891609],
        [-0.0071458369,  0.0496945158],
        [-0.0419657491, -0.0426752158],
        [ 0.0149004273, -0.0379221700],
        [-0.0320366025,  0.0642739236],
        [-0.0018672049, -0.0085368231],
        [ 0.0299312249,  0.0012228340]])

In [ ]:
y, linear_predictor, time, event = ah_data_2(type='np')
grad_own, hess_own = ah_objective(
    y, linear_predictor
)
grad_own

array([ 0.02956914,  0.00345863,  0.00275022,  0.00240579, -0.00714583,
       -0.04196575,  0.01490039, -0.03203659, -0.0018672 ,  0.02993121])

### Gradient Function Comparison

In [ ]:
np.allclose(grad_own, ah_grad_paper[:,0])

True

## Hessian Function Comparison

In [ ]:

from torch.autograd.functional import hessian as hess_torch
y, linear_predictor, time, event = ah_data_2(type='torch')
print(y.shape, linear_predictor.shape)
hessian_matrix = hess_torch(ah_likelihood_torch, (linear_predictor, y), create_graph=True)
diag_hessian = hessian_matrix[0][0].diag()
diag_hessian


torch.Size([10]) torch.Size([10])


tensor([ 0.1197987571,  0.0018476250, -0.0024721003,  0.0112646371,
         0.0286145564,  0.0186578501, -0.0733677074,  0.0219786447,
        -0.0648421198, -0.0534176268], grad_fn=<DiagBackward0>)

In [ ]:
y, linear_predictor, time, event = ah_data_2(type='np')
grad_own, hess_own = ah_objective(
    y, linear_predictor
)
hess_own

array([0.1671356 , 0.09057899, 0.08653509, 0.10661001, 0.1241162 ,
       0.18680498, 0.242457  , 0.08737591, 0.30874032, 0.27100193])

In [ ]:
diag_hessian

tensor([2.])